In [1]:
from __future__ import print_function
import numpy as np
import random

class Learner:
    Q = np.random.rand(3,3,3,3,3,3,3,3,3,9) 
    #Q = np.zeros((3,3,3,3,3,3,3,3,3,9))
    age = 1    

    def __init__(self, exploitRate=.9):
        self.eta = 0.7 #learning rate
        self.gamma = .7 #discount rate 
        self.softmax = False
        self.exploitRate = exploitRate
        
        self.oldBoard = tuple(np.zeros((3,3), dtype=np.int).ravel())
        self.chosenMove = 0
    
    def thinkMove(self, board):
        self.oldBoard = tuple(board.ravel())
        if self.softmax:
            Qmoves = Learner.Q[self.oldBoard]
            P = np.exp(Qmoves * self.exploitRate * Learner.age)
            Learner.age += 1
            P = P / np.sum(P)
            self.chosenMove = np.random.choice(np.arange(0,9), p=P)
        else:
            if random.random() < self.exploitRate:
                self.chosenMove = np.argmax(Learner.Q[self.oldBoard])
            else:
                self.chosenMove = np.random.choice(np.arange(0,9))
        return (self.chosenMove/3, self.chosenMove%3)
    
    def updateQ(self, board, gameState, learnsTurn):
        r = 0
        if gameState == 'l' and learnsTurn:
            r = -1.
        elif gameState == 'w' and learnsTurn:
            r = 1.
        elif gameState == 'w' and not learnsTurn:
            r = -1.
        Learner.Q[self.oldBoard][self.chosenMove] = (1-self.eta)*Learner.Q[self.oldBoard][self.chosenMove] + \
        self.eta*(r + self.gamma*(np.amax(Learner.Q[tuple(board.ravel())])))
        
        #print(np.asarray(self.oldBoard).reshape(3,3), self.chosenMove)
        #print(Learner.Q[self.oldBoard][self.chosenMove])
        return
        

In [2]:

def makeMove((row, col), player):
    #check oob
    if row!=0 and row!=1 and row!=2:
        return 'l'
    if col!=0 and col!=1 and col!=2:
        return '1'
    if board[row][col] != 0:
        return 'l'
    #make the move
    board[row][col] = player
    
    #print(board)
    #print("")
    
    #check for win
    if board[0][col] != 0 and board[0][col] == board[1][col] and board[1][col] == board[2][col]:
        return 'w'
    if board[row][0] != 0 and board[row][0] == board[row][1] and board[row][1] == board[row][2]:
        return 'w'
    if board[0][0] != 0 and board[0][0] == board[1][1] and board[1][1] == board[2][2]:
        return 'w'
    if  board[2][0] != 0 and board[2][0] == board[1][1] and board[1][1] == board[0][2]:
        return 'w'
    if (board == np.zeros((3,3), dtype=np.int)).sum() == 0:
        return 't'
    return 'c'

def aiRandom():
    moves = []
    for i in range(0,3):
        for j in range(0,3):
            if board[i][j] == 0:
                moves.append((i,j))
    if len(moves) == 0:
        return (-1,-1)
    return random.choice(moves)

def aiHuman():
    print(board)
    print("")
    
    row = input()
    col = input()
    return (row, col)

def playGame(learnerFirst, learner):
    learnsTurn = False
    turnN = 0
    gameState = 'c'
    
    if not learnerFirst:
        gameState = makeMove(aiRandom(), 1)
        turnN =+ 1
        
    while gameState == 'c':
        gameState = makeMove(learner.thinkMove(board), 2)
        turnN += 1
        if gameState != 'c':
            learnsTurn = True
            learner.updateQ(board, gameState, learnsTurn)
            break
        gameState = makeMove(aiRandom(), 1)
        turnN += 1
        learner.updateQ(board, gameState, learnsTurn)
    
    #print(gameState + " aiLearn" if learnsTurn else gameState + " aiRandom")
    return (gameState, learnsTurn)
        

learner1 = Learner(exploitRate=1)
learnerWins = 0
challengerWins = 0
ties = 0

for i in range(0,10000):
    board = np.zeros((3,3), dtype=np.int)
    #res = playGame(i%2, learner1)
    res = playGame(True, learner1)
    
    if res[0] == 't':
        ties += 1
    elif res == ('w', True):
        learnerWins += 1
    elif res == ('l', False):
        learnerWins += 1
    elif res == ('l', True):
        challengerWins += 1
    elif res == ('w', False):
        challengerWins += 1
        
print(learnerWins, challengerWins, ties)




    

SyntaxError: invalid syntax (<ipython-input-2-c487152b9704>, line 2)

In [ ]:
indeces = (2,2,1,
           1,2,2,
           2,1,1)
print(Learner.Q[indeces])
print(np.argmax(Learner.Q[indeces]))